<a href="https://colab.research.google.com/github/JTT94/nlp_neural_process/blob/master/BERT_NeuralProcesses_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
import os
import re

from keras import backend as K
import numpy as np
import string
from datetime import datetime 
import tensorflow_probability as tfp
from tensorflow_probability import distributions as tfd

W0519 17:29:10.070685 140114523699072 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14
Using TensorFlow backend.



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
!pip install bert-tensorflow
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization


     |████████████████████████████████| 71kB 1.5MB/s 


In [0]:
# initialiase tensorboard 
# from https://stackoverflow.com/questions/47818822/can-i-use-tensorboard-with-google-colab


# Get TensorBoard running in the background. 
LOG_DIR = './test_output'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [4]:
# # #Download and unzip ngrok. 
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip


--2019-05-19 17:29:46--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.207.111.186, 34.206.130.40, 52.45.111.123, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.207.111.186|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16648024 (16M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  15.88M  38.6MB/s    in 0.4s    

2019-05-19 17:29:46 (38.6 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [16648024/16648024]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
# #Launch ngrok background process...
get_ipython().system_raw('./ngrok http 6006 &')

In [6]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://c7d20d96.ngrok.io


In [7]:
import os, sys
sys.path.append('../') # add personal code dir to path for import


!test -d neural_process || git clone https://github.com/JTT94/nlp_neural_process.git neural_process
sys.path.append('./neural_process/')

import random
from neural_process import split_context_target, NeuralProcessParams
from neural_process.network import *
from neural_process.loss import *
from neural_process.predict import *
from neural_process.process import *

from neural_process.tf_model_builder_AUC import *
from neural_process.bert_utils import *

Cloning into 'neural_process'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 42 (delta 17), reused 26 (delta 12), pack-reused 0
Unpacking objects: 100% (42/42), done.


In [8]:
# Initialize session
sess = tf.Session()
K.set_session(sess)
K.tensorflow_backend._get_available_gpus()
# tf.logging.set_verbosity(tf.logging.ERROR)

['/job:localhost/replica:0/task:0/device:GPU:0']

In [9]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [10]:
# filename = './cleaned_data.csv'
# filename = './data1.csv'
# df = pd.read_csv(filename, index_col=0)
# cols = ['comment','antagonize', 'condescending', 'dismissive', 'generalisation', 'hostile', 'sarcastic', 'healthy']
# cols = ['comment','cleaned_comment','antagonize', 'condescending', 'dismissive', 'generalisation', 'hostile', 'sarcastic', 'healthy']
# df = df[cols]

#score_column = ['antagonize', 'condescending', 'dismissive', 'generalisation', 'hostile', 'sarcastic', 'healthy']
# text_col_name = 'cleaned_comment'
# text_col_name = 'comment'

#--------

## For kaggle dataset

filename = './gdrive/My Drive/Data/train.csv'
df = pd.read_csv(filename)
cols = ['comment_text','toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
df = df[cols]

score_column = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'] 

text_col_name = 'comment_text'

#Cast to float - because scores and labels need to be concattenated in the model function, and so need to be same type
for i in score_column:
  df[i] = pd.to_numeric(df[i],downcast='float')
  
len(df)

159571

In [11]:
# Restrict comment length

df = df[df.comment_text.str.len() <= 250]

# print(len(df))

# full_df = df.copy(deep=True)

#sample portion of dataset to speed up preprocessing

# df = df.sample(frac = 0.5)

# over represent toxic comments

df['num_toxic_atts'] = df[cols[1:]].apply(lambda x: np.sum(x), axis = 1)

df_toxic = df[df.num_toxic_atts  > 0]
# print(len(df_toxic))
df_healthy_sample = df[df.num_toxic_atts == 0][:len(df_toxic)]
print(len(df_healthy_sample))
df_healthy_remaining = df[df.num_toxic_atts == 0][len(df_toxic):]
print(len(df_healthy_remaining))

df_overrep = pd.concat([df_toxic, df_healthy_sample]).sample(frac=1.0)
print(len(df_overrep))
train_propn = 0.8

ratio = len(df_toxic)/(len(df_healthy_sample)+len(df_healthy_remaining))
print(ratio)

df_train = df_overrep[:int(len(df_overrep)*train_propn)]
print(len(df_train))

#construct a test set with similar proportion/imbalance in toxic - nontoxic data to the real data set
df_test = pd.concat([df_overrep[int(len(df_overrep)*train_propn):], df_healthy_remaining[:int((1-0.8)*(len(df_healthy_remaining)-len(df_train)))]])

print(0.5*len(df_overrep[int(len(df_overrep)*train_propn):]) / len(df_test))
# df_train.head(50)

11640
67417
23280
0.14723553891496008
18624
0.16150964340224783


In [12]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_model_hub = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
tokenizer = create_tokenizer_from_hub_module(BERT_model_hub)

Instructions for updating:
Colocations handled automatically by placer.


W0519 17:33:59.850676 140114523699072 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0519 17:34:02.017913 140114523699072 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [0]:
# #Pre process data for bert embedding

max_seq_length = 128

import pickle

train_input_examples = create_examples(df_train, score_column, text_col_name)
test_input_examples = create_examples(df_test, score_column, text_col_name)

train_features = convert_examples_to_features(train_input_examples, max_seq_length, tokenizer)
test_features = convert_examples_to_features(test_input_examples, max_seq_length, tokenizer)

pickle.dump(train_features, open('./gdrive/My Drive/Data/train_features.p', 'wb'))
pickle.dump(test_features, open('./gdrive/My Drive/Data/test_features.p', 'wb'))

# # ## Load features previously saved

# train_features = pickle.load(open('./gdrive/My Drive/Kaggle_toxic_comments/train_features.p', 'rb'))
# test_features = pickle.load(open('./gdrive/My Drive/Kaggle_toxic_comments/train_features.p', 'rb'))

In [0]:

def model_fn_builder(create_model, num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
    """Returns `model_fn` closure for TPUEstimator."""
    def model_fn(features, mode, params):  # pylint: disable=unused-argument
        """The `model_fn` for TPUEstimator."""

        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]    
        scores = features["scores"]
        
        
        (loss, posterior_predict, prior_predict, true_y) = create_model(input_ids, input_mask, segment_ids, num_labels, scores)
        
          
        train_op = bert.optimization.create_optimizer(loss, learning_rate, num_train_steps, 
            num_warmup_steps, use_tpu=False)
        
        # output from model
        ystar, variance = tf.nn.moments(posterior_predict.mu,[0])
        
        # Calculate evaluation metrics.
        eval_metrics = {}
        
        # AUC
        def metric_fn(pred_scores, real_scores, trait_num):
            auc_value = tf.metrics.auc(real_scores[:,trait_num], pred_scores[:,trait_num])
            accuracy_value = tf.metrics.accuracy(labels = tf.round(real_scores[:,trait_num]), predictions=tf.round(pred_scores[:,trait_num]))
            return {"auc"+str(trait_num): auc_value, "accuracy"+str(trait_num): accuracy_value}

        labels = true_y # need to round them if true labels are not 1 or 0
        eval_metrics_lst = [metric_fn(ystar, labels, trait_num) for trait_num in range(num_labels)]
        
        for d in eval_metrics_lst:
            tf.summary.scalar(list(d.keys())[0], list(d.values())[0][1]) #make available to tensorboard
            eval_metrics.update(d)
            
        # accuracy
        #correct_prediction = tf.equal(tf.round(tf.nn.sigmoid(ystar)), tf.round(labels))
        #overall_accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        #eval_metrics.update({'overall_accuracy':overall_accuracy})
        
        #accuracy_by_label = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), 0)
        #eval_metrics.update({'accuracy_by_label':accuracy_by_label})
        
        # returns 
        if mode == tf.estimator.ModeKeys.TRAIN:
            return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
        elif mode == tf.estimator.ModeKeys.PREDICT:
            return tf.estimator.EstimatorSpec(mode=mode, predictions={'prediction_mean': ystar, 'prediction_var': variance})
        else:
            return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metrics)
        # Return the actual model function in the closure
    return model_fn

In [0]:
outputDim = len(score_column)
num_draws = 20

def encoder_h(context_xys: tf.Tensor, params: NeuralProcessParams) -> tf.Tensor:
    """Map context inputs (x_i, y_i) to r_i

    Creates a fully connected network with a single sigmoid hidden layer and linear output layer.

    Parameters
    ----------
    context_xys
        Input tensor, shape: (n_samples, dim_x + dim_y)
    params
        Neural process parameters

    Returns
    -------
        Output tensor of encoder network
    """
    hidden_layer = context_xys
#     print('hidden layer')
#     print(hidden_layer)
#     print(enumerate(params.n_hidden_units_h))
    # First layers are relu
    for i, n_hidden_units in enumerate(params.n_hidden_units_h):
#         print(i)
#         print(n_hidden_units)
        hidden_layer = tf.layers.dense(hidden_layer, n_hidden_units,
                                       activation=tf.nn.relu,
                                       name='encoder_layer_{}'.format(i),
                                       reuse=tf.AUTO_REUSE,
                                       kernel_initializer='normal')
#         print(hidden_layer)

    # Last layer is simple linear
    i = len(params.n_hidden_units_h)
    r = tf.layers.dense(hidden_layer, params.dim_r,
                        name='encoder_layer_{}'.format(i),
                        reuse=tf.AUTO_REUSE,
                        kernel_initializer='normal')
    return r
  
  
def aggregate_r(context_rs: tf.Tensor) -> tf.Tensor:
    """Aggregate the output of the encoder to a single representation

    Creates an aggregation (mean) operator to combine the encodings of multiple context inputs

    Parameters
    ----------
    context_rs
        Input encodings tensor, shape: (n_samples, dim_r)

    Returns
    -------
        Output tensor of aggregation result
    """
    mean = tf.reduce_mean(context_rs, axis=0)

    r = tf.reshape(mean, [1, -1])
    return r


def get_z_params(context_r: tf.Tensor, params: NeuralProcessParams) -> GaussianParams:
    """Map encoding to mean and covariance of the random variable Z

    Creates a linear dense layer to map encoding to mu_z, and another linear mapping + a softplus activation for Sigma_z

    Parameters
    ----------
    context_r
        Input encoding tensor, shape: (1, dim_r)
    params
        Neural process parameters

    Returns
    -------
        Output tensors of the mappings for mu_z and Sigma_z
    """
    mu = tf.layers.dense(context_r, params.dim_z, name="z_params_mu", reuse=tf.AUTO_REUSE, kernel_initializer='normal')

    sigma = tf.layers.dense(context_r, params.dim_z, name="z_params_sigma", reuse=tf.AUTO_REUSE,
                            kernel_initializer='normal')
    sigma = tf.nn.softplus(sigma)

    return GaussianParams(mu, sigma)


def decoder_g(input_xs_embedding, z_samples: tf.Tensor, params: NeuralProcessParams,
              noise_std: float = 0.05) -> GaussianParams:
    """Determine output y* by decoding input and latent variable

    Creates a fully connected network with a single sigmoid hidden layer and linear output layer.

    Parameters
    ----------
    z_samples
        Random samples from the latent variable distribution, shape: (n_z_draws, dim_z)
    input_xs
        Input values to predict for, shape: (n_x_samples, dim_x)
    params
        Neural process parameters
    noise_std
        Constant standard deviation used on output

    Returns
    -------
        Output tensors for the parameters of Gaussian distributions for target outputy, where its mean mu has shape
        (n_x_samples, n_z_draws)
 
    """
    # inputs dimensions
    # z_sample has dim [n_draws, dim_z]
    # x_star has dim [N_star, dim_x]

    n_draws = z_samples.get_shape().as_list()[0]
#     print('n_draws')
#     print(n_draws)
    n_xs = tf.shape(input_xs_embedding)[0]

    # Repeat z samples for each x*
    z_samples_repeat = tf.expand_dims(z_samples, axis=1)
    z_samples_repeat = tf.tile(z_samples_repeat, [1, n_xs, 1])

    # Repeat x* for each z sample
#     input_xs_embedding = embedder(input_xs)
    x_star_repeat = tf.expand_dims(input_xs_embedding, axis=0)
    x_star_repeat = tf.tile(x_star_repeat, [n_draws, 1, 1])

    # Concatenate x* and z
    # shape: (n_z_draws, n_xs, dim_x + dim_z)
    inputs = tf.concat([x_star_repeat, z_samples_repeat], axis=2)

    hidden_layer = inputs
    # First layers are relu
    for i, n_hidden_units in enumerate(params.n_hidden_units_g):
        hidden_layer = tf.layers.dense(hidden_layer, n_hidden_units,
                                       activation=tf.nn.relu,
                                       name='decoder_layer_{}'.format(i),
                                       reuse=tf.AUTO_REUSE,
                                       kernel_initializer='normal')

    # Last layer is simple linear
    i = len(params.n_hidden_units_g)
    hidden_layer = tf.layers.dense(hidden_layer, outputDim,
                                   name='decoder_layer_{}'.format(i),
                                   reuse=tf.AUTO_REUSE,
                                   kernel_initializer='normal')

    hidden_layer = tf.math.sigmoid(hidden_layer)
    
    # mu will be of the shape [N_star, n_draws]
    mu_star = hidden_layer

    sigma_star = tf.constant(noise_std, dtype=tf.float32)

    return GaussianParams(mu_star, sigma_star)


def xy_to_z_params(context_xs: tf.Tensor, context_ys: tf.Tensor,
                   params: NeuralProcessParams) -> GaussianParams:
    """Wrapper to create full network from context samples to parameters of pdf of Z

    Parameters
    ----------
    context_xs
        Tensor with context features, shape: (n_samples, dim_x)
    context_ys
        Tensor with context targets, shape: (n_samples, dim_y)
    params
        Neural process parameters

    Returns
    -------
        Output tensors of the mappings for mu_z and Sigma_z
    """
#     context_xs = embedder(context_xs)
    xys = tf.concat([context_xs, context_ys], axis=1)
    rs = encoder_h(xys, params)
    r = aggregate_r(rs)
    z_params = get_z_params(r, params)
    return z_params

def loglikelihood(y_star: tf.Tensor, y_pred_params: GaussianParams):
    """Log-likelihood of an output given a predicted """
    p_normal = tfd.MultivariateNormalDiag(loc=y_pred_params.mu)
    loglike = p_normal.log_prob(y_star)
    loglike = tf.reduce_sum(loglike, axis=0)
    loglike = tf.reduce_mean(loglike)
    return loglike
  
  
def create_model(input_ids, input_mask, segment_ids, num_labels, scores):

#     """Creates a classification model."""

    # 1 split input ids, input mask, seg ids and scores into context and target
#     tf.logging.info('input ids shape')
      
    
    bert_module = hub.Module(BERT_model_hub,trainable=True)

    bert_inputs = dict(input_ids=input_ids,input_mask=input_mask,
      segment_ids=segment_ids)

    bert_outputs = bert_module(inputs=bert_inputs,signature="tokens",
      as_dict=True)
    
  # Use "pooled_output" for classification tasks on an entire sentence. Use "sequence_outputs" for token-level output.
    output_bert_layer = bert_outputs["pooled_output"]

    tf.logging.info(output_bert_layer)
    
    hidden_size = output_bert_layer.shape[-1].value

    # -------
     
    params = NeuralProcessParams(dim_r=20, dim_z=20, n_hidden_units_h=[128, 128, 128], n_hidden_units_g=[128, 128, 128])

    btch_sz = tf.shape(output_bert_layer)[0]
    
    n_context = tf.random_shuffle(tf.range(1,btch_sz))[0]
    
    indices = tf.range(0, btch_sz)
    context_set_indices = tf.gather(tf.random_shuffle(indices),tf.range(n_context))
    target_set_indices = tf.gather(tf.random_shuffle(indices),tf.range(n_context, btch_sz))
    
    context_xs = tf.gather(output_bert_layer,context_set_indices)
    context_ys = tf.gather(scores, context_set_indices)
    target_xs = tf.gather(output_bert_layer,target_set_indices)
    target_ys = tf.gather(scores,target_set_indices)

    x_all = output_bert_layer
    y_all = scores
    
    z_context = xy_to_z_params(context_xs, context_ys, params)
    z_all = xy_to_z_params(x_all, y_all, params)
    
    epsilon = tf.random_normal([num_draws, params.dim_z])
    z_samples = tf.multiply(epsilon, z_all.sigma)
    z_samples = tf.add(z_samples, z_all.mu)
    
    
    y_loss_params = decoder_g(target_xs, z_samples, params)
    
    prior_predict = decoder_g(epsilon, z_samples, params)
    posterior_predict = decoder_g(x_all, z_samples, params)

    loglike = loglikelihood(target_ys, y_loss_params)
    KL_loss = KLqp_gaussian(z_all.mu, z_all.sigma, z_context.mu, z_context.sigma)
    loss = tf.negative(loglike) + KL_loss
    
    return (loss, posterior_predict, prior_predict, y_all)


In [0]:
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
batch_size = 32
# lr = 2e-2
lr = 2e-5
epochs = 6.0
# Warmup is a period of time where the learning rate  is small and gradually increases
warmpup_proportion = 0.1

# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / batch_size *epochs)
num_warmup_steps = int(num_train_steps * warmpup_proportion)

#####

output_dir = "./test_output"
save_checkpoints_steps = 500
save_summary_steps = 100

# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(model_dir=output_dir,
    save_summary_steps=save_summary_steps, save_checkpoints_steps=save_checkpoints_steps)

#####

num_labels = len(score_column)

model_fn = model_fn_builder(create_model, num_labels = num_labels, learning_rate=lr,
  num_train_steps=num_train_steps, num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(model_fn=model_fn,config=run_config,
  params={"batch_size": batch_size})

#####

# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = input_fn_builder(
    features=train_features, seq_length=max_seq_length, 
    num_labels = num_labels, is_training=True, drop_remainder=False)

#####

print('Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

INFO:tensorflow:Using config: {'_model_dir': './test_output', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6e512e3320>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0519 22:25:34.467746 140114523699072 estimator.py:201] Using config: {'_model_dir': './test_output', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6e512e3320>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Beginning Training!
INFO:tensorflow:Calling model_fn.


I0519 22:25:44.437846 140114523699072 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

I0519 22:57:16.539145 140114523699072 basic_session_run_hooks.py:247] loss = 110.54612, step = 2110 (84.823 sec)


INFO:tensorflow:global_step/sec: 1.18177


I0519 22:58:41.152578 140114523699072 basic_session_run_hooks.py:680] global_step/sec: 1.18177


INFO:tensorflow:loss = 112.35468, step = 2210 (84.618 sec)


I0519 22:58:41.157131 140114523699072 basic_session_run_hooks.py:247] loss = 112.35468, step = 2210 (84.618 sec)


In [0]:
eval_steps = int(len(df_test) / batch_size)
eval_input_fn = input_fn_builder(
    features=test_features,
    seq_length=max_seq_length,
    num_labels = num_labels,
    is_training=False,
    drop_remainder=True)
result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)

In [221]:
print("***** Eval results *****")
for key in sorted(result.keys()):
  print('  {} = {}'.format(key, str(result[key])))
      

***** Eval results *****
  accuracy0 = 0.46875
  accuracy1 = 0.9625
  accuracy2 = 0.74375
  accuracy3 = 0.99375
  accuracy4 = 0.775
  accuracy5 = 0.95
  auc0 = 0.6247059
  auc1 = 0.4383118
  auc2 = 0.4928264
  auc3 = 0.80503166
  auc4 = 0.4919355
  auc5 = 0.45394742
  global_step = 10
  loss = 124.019165


In [0]:
predict_features = test_features[:10]

In [0]:
test_input_fn = input_fn_builder(
    features=predict_features, seq_length=max_seq_length, 
    num_labels = num_labels, is_training=False, drop_remainder=False)

In [0]:
test_scores = np.array([predict_features[i].score for i, val in enumerate(predict_features)])

preds = estimator.predict(input_fn=test_input_fn)
preds_vals = list(preds)
pred_means = np.array([val['prediction_mean'] for key, val in enumerate(preds_vals)])

In [0]:
def total_accuaracy(pred_means, test_scores):
  preds = np.round(pred_means)
  correct_guesses = np.reshape(preds == test_scores,-1)
  return np.sum(correct_guesses) / len(correct_guesses)

In [0]:
def class_accuaracy(pred_means, test_scores):
  preds = np.round(pred_means)
  correct_guesses = preds == test_scores
  return correct_guesses.sum(axis=0)/ len(correct_guesses)


In [209]:
print(total_accuaracy(pred_means, test_scores))
print(class_accuaracy(pred_means, test_scores))

0.8333333333333334
[0.7 1.  0.6 1.  0.7 1. ]
